In [1]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
import pickle
from datetime import date,datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score 
import time
import matplotlib.pyplot as plt
from sklearn.utils import resample
import datetime

In [2]:
df = pd.read_pickle(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN\Data Files\data_dump_income_pred.pkl")

In [10]:
df[df.TIME_DECISION_DATE == datetime.datetime(2019,1,1)].head()

,SKP_CREDIT_CASE,TIME_DECISION_DATE,CUS_LAT,CUS_LONG,RISK_FPD10,RISK_FPD30,RISK_FSPD30,RISK_FSTPD30,RISK_FSTQPD30,RISK_FSTQPD60,NTC_FLAG,GRID_ID,RISK_SCORE,Unnamed: 0,CLIENT_INR_INCOME,CLIENT_DEBT,CLIENT_EMI
2683029,220543872,2019-01-01,26.196128,73.002229,0.0,0.0,0.0,0.0,0.0,0.0,Others,500_01007_04165,0.037345,2340213,20000.0,0,1343.0
2683030,220543873,2019-01-01,26.280459,72.987023,0.0,0.0,0.0,0.0,0.0,0.0,Others,500_01005_04183,0.016066,2340214,18000.0,0,1430.0
2683031,220543874,2019-01-01,21.116755,79.104340,0.0,0.0,0.0,0.0,0.0,0.0,NTC,500_02197_03028,0.054663,2340215,16000.0,0,1715.0
2683032,219906800,2019-01-01,18.637497,73.836025,0.0,0.0,0.0,0.0,0.0,0.0,NTC,500_01099_02515,0.111290,2273269,18000.0,0,2608.0
2683033,219993472,2019-01-01,23.194511,77.462190,0.0,0.0,0.0,0.0,0.0,0.0,Others,500_01870_03483,NaN,2278445,15000.0,0,1343.0


In [27]:
df = df[["SKP_CREDIT_CASE","TIME_DECISION_DATE","CUS_LAT","CUS_LONG","RISK_FSTQPD30",
"RISK_SCORE","GRID_ID","CLIENT_INR_INCOME"]]

In [28]:
df.dropna(subset = ["RISK_SCORE"], inplace = True)

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
df["MONTH_STR"] = df["TIME_DECISION_DATE"].dt.strftime("%Y-%m")

C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
df = df[df["CLIENT_INR_INCOME"]<df["CLIENT_INR_INCOME"].quantile(0.99)]

In [31]:
df_final_test = pd.DataFrame()
seed = 26

import datetime
from sklearn.neighbors import KNeighborsRegressor

for x in np.arange(1,9,1):
    
    date_test_start = datetime.datetime(2019, x, 1)
    date_test_end = (date_test_start +  datetime.timedelta(days=31)).replace(day=1)
    date_train_start = (date_test_start-datetime.timedelta(days=1)-datetime.timedelta(days=180)) 
    date_train_end = (date_test_start-datetime.timedelta(days=1))
    
    start_time = datetime.datetime.now()
    df_train = df[(df["TIME_DECISION_DATE"]>=date_train_start) & 
              (df["TIME_DECISION_DATE"]<date_train_end)]
    df_test = df[(df["TIME_DECISION_DATE"]>=date_test_start) & 
                  (df["TIME_DECISION_DATE"]<date_test_end)]

    
    df_train= df_train.sample(n=800000, random_state=seed)
    
    X_train = df_train[["CUS_LAT","CUS_LONG","TIME_DECISION_DATE"]]
    X_test = df_test[["CUS_LAT","CUS_LONG","TIME_DECISION_DATE"]]

    y_train = df_train["CLIENT_INR_INCOME"]
    y_test =  df_test["CLIENT_INR_INCOME"]

    knn = KNeighborsRegressor(n_neighbors=201)
    bst = knn.fit(X_train[["CUS_LAT","CUS_LONG"]], np.ravel(y_train))

    ypred_train = bst.predict(X_train[["CUS_LAT","CUS_LONG"]])
    ypred_test = bst.predict(X_test[["CUS_LAT","CUS_LONG"]])

    df_test['PRED_INCOME'] = ypred_test
    
    if df_final_test.shape == (0,0):
        df_final_test = df_test
    else:
        df_final_test = pd.concat([df_final_test,df_test],ignore_index=True)

    print("Training for predicting (month):",date_test_start.month, "    RMSE TRAIN:",np.sqrt(metrics.mean_squared_error(y_train,ypred_train)), "    RMSE TEST:",np.sqrt(metrics.mean_squared_error(y_test,ypred_test)),
         "    time taken: ",'{:,.2}'.format((datetime.datetime.now()-start_time).seconds/60))




C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 1     RMSE TRAIN: 5609.251634645549     RMSE TEST: 5448.911649638455     time taken:  2.2


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 2     RMSE TRAIN: 5591.368589800107     RMSE TEST: 5341.325228060757     time taken:  2.0


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

Training for predicting (month): 6     RMSE TRAIN: 5522.009740438813     RMSE TEST: 5889.695531456717     time taken:  2.1


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 7     RMSE TRAIN: 5596.834178096526     RMSE TEST: 5900.281422325711     time taken:  2.1


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Training for predicting (month): 8     RMSE TRAIN: 5661.6425312601905     RMSE TEST: 5845.3827892116105     time taken:  2.1


In [35]:
df_final_test.MONTH_STR.value_counts()

2019-07    337862
2019-05    337501
2019-06    336452
2019-08    310439
2019-01    297066
2019-03    280988
2019-04    270441
2019-02    257090
Name: MONTH_STR, dtype: int64

In [36]:
df_final_test.shape

(2427839, 10)

In [37]:
df_final_test.to_pickle(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN\Data Files\scored_income_2019.pkl")